In [ ]:
%pip install datasets evaluate
%pip install transformers[torch]
%pip install accelerate -U
%pip3 install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.9 MB/s eta 0:00:00


UsageError: Line magic function `%pip3` not found.


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv(
    "https://www.dropbox.com/s/20yp4jzq7de0wzf/train.csv?dl=1"
)
print(df.head())

   id keyword location                                               text  \
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...   
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada   
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...   
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...   
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...   

   target  
0       1  
1       1  
2       1  
3       1  
4       1  


In [ ]:
# Shuffling and dropping unnecessary columns
df_shuffled = df.sample(frac=1, random_state=42)

# Dropping id, keyword and location columns as these columns consists of mostly nan values
# we will be using only text and target columns
df_shuffled.drop(["id", "keyword", "location"], axis=1, inplace=True)
df_shuffled.reset_index(inplace=True, drop=True)
print(df_shuffled.head())

                                                text  target
0  So you have a new weapon that can cause un-ima...       1
1  The f$&amp;@ing things I do for #GISHWHES Just...       0
2  DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...       1
3  Aftershock back to school kick off was great. ...       0
4  in response to trauma Children of Addicts deve...       0


In [ ]:
test_df = df_shuffled.sample(frac=0.1, random_state=42)
train_df = df_shuffled.drop(test_df.index)
print(f"Using {len(train_df)} samples for training and {len(test_df)} for validation")

Using 6852 samples for training and 761 for validation


In [ ]:
train_df = train_df.to_numpy()
test_df  = test_df.to_numpy()

In [ ]:
train_df[0][0]

'So you have a new weapon that can cause un-imaginable destruction.'

In [ ]:
from transformers import AutoTokenizer

pretrain_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
import transformers
print(transformers.__version__)

4.38.2


In [ ]:
import torch
print(torch.__version__)

2.1.0+cu121


In [ ]:
import torchtext
print(torchtext.__version__)

0.16.0+cpu


In [ ]:
import torch
from torch.utils.data import Dataset


class Tweets_Dataset(Dataset):
    def __init__(self, data, max_len,  tokenizer):
        super().__init__()
        self.data = data
        self.max_len = max_len
        self.tokenizer = tokenizer

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        text = self.data[idx][0]
        label = self.data[idx][1]

        text_processed = self.tokenizer.encode(text, max_length=self.max_len, padding='max_length')

        return {
            "input_ids": text_processed,
            "labels": label
        }

In [ ]:
max_length = 200
train_data = Tweets_Dataset(train_df, max_length, pretrain_tokenizer)
test_data = Tweets_Dataset(test_df, max_length, pretrain_tokenizer)

In [ ]:
id2label = {0: "POSITIVE", 1: "NEGATIVE"}
label2id = {"POSITIVE": 0, "NEGATIVE": 1}

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",
                                                           num_labels=2,
                                                           id2label=id2label,
                                                           label2id=label2id)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer


training_args = TrainingArguments(
    output_dir="out_dir",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer = pretrain_tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.495334,0.754271
2,0.533400,0.445283,0.814717
3,0.380200,0.498063,0.804205
4,0.304200,0.466676,0.804205
5,0.248200,0.576272,0.804205
6,0.203900,0.617560,0.798949
7,0.178100,0.656413,0.806833
8,0.178100,0.835081,0.802891
9,0.138000,0.987274,0.800263
10,0.105400,1.001656,0.812089


TrainOutput(global_step=8580, training_loss=0.15139906698173577, metrics={'train_runtime': 2698.4115, 'train_samples_per_second': 50.785, 'train_steps_per_second': 3.18, 'total_flos': 7091145434304000.0, 'train_loss': 0.15139906698173577, 'epoch': 20.0})

In [ ]:
trainer.evaluate(test_data)

{'eval_loss': 0.4452829957008362,
 'eval_accuracy': 0.8147174770039421,
 'eval_runtime': 4.609,
 'eval_samples_per_second': 165.113,
 'eval_steps_per_second': 10.414,
 'epoch': 20.0}